In [1]:
import numpy  as np
import matplotlib.pyplot as plt

In [2]:
def bracket_livro(x, y, k = 2, l_r=None, x1=None, x2=None):

    # Iniciar coordenadas caso não sejão específicadas 
    if x1==None:
        x1 = x[0]
    if x2==None:
        x2 = x[-1]

    # Iniciar o parâmetro condizente ao tamanho do passo
    if l_r == None:
        l_r = 0.01

    if y(x1) > y(x2):
        x1 = x2

    # Variável auxiliar 
    xx2 = x2
    while True:
        # Executando os passos
        xx2 = xx2 - xx2 * l_r

        ''' Se o passo atual resulta em uma função 
        maior que a anterior a otimização é encerrada'''
        if y(x2) <= y(xx2):
            x2 = xx2

            # Case para retornar a sequencia menor para maior
            if x1 > x2:
                return x2, x1
            return x1, x2
        
        x2 = xx2